# MSPMA(Maximum Sales Probability Matching Algorithm)

# 2020년 6월 1일의 시간대별 상품 취급액 예측 후 MSPMA로 새로운 편성 확인

In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import random
import sys
import re
import matplotlib.pyplot as plt

from sklearn.preprocessing import RobustScaler

from keras.layers import Dense, Activation, Flatten, Conv1D, MaxPooling1D, AveragePooling1D, GlobalMaxPooling1D, Conv1DTranspose, concatenate, Input, add, Dropout, BatchNormalization
from tensorflow.keras import Model

In [ ]:
# 숫자 표기법 변환
pd.set_option('display.float_format', '{:.2f}'.format)

# 한글 깨짐 방지
plt.rc('font', family='Malgun Gothic')

# 도화지 크기 설정
plt.rcParams['figure.figsize'] = (8, 8)

## 1. 테스트 데이터의 6월 1일만 추출하여 데이터 셋 만들기

In [ ]:
org_test = pd.read_csv("/content/drive/Shared drives/빅콘테스트2020/data/data/02_평가데이터/test.csv", header=1)

In [ ]:
test = pd.read_csv('/content/drive/Shared drives/빅콘테스트2020/real_test.csv')

In [ ]:
total_test = pd.concat([org_test, test], axis=1)

In [ ]:
# 테스트 데이터에서 2020년 6월 1일의 row는 06시부터 02시(다음날이지만 한 싸이클로 포함)까지 79개
june_1st = total_test.iloc[:79, :]

In [ ]:
# 판매단가가 '-'인 것 0으로 채우기
june_1st['판매단가'] = ['0' if x == ' - ' else x for x in june_1st['판매단가']]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
# 판매단가 integer 타입으로 변환
june_1st['판매단가'] = [x.replace(',','').strip() for x in june_1st['판매단가']]
june_1st['판매단가'] = june_1st['판매단가'].astype('int')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
# 표시된 것 이외에는 1시간에 3개 row씩 존재
product1 = june_1st.iloc[:3,:]
product2 = june_1st.iloc[3:6,:]
product3 = june_1st.iloc[6:9,:]
product4 = june_1st.iloc[9:12,:]
product5 = june_1st.iloc[12:15,:]
product6 = june_1st.iloc[15:18,:]
product7 = june_1st.iloc[18:21,:]
product8 = june_1st.iloc[21:27,:] # 6개
product9 = june_1st.iloc[27:28,:] # 1개
product10 = june_1st.iloc[28:31,:]
product11 = june_1st.iloc[31:34,:]
product12 = june_1st.iloc[34:40,:] # 6개
product13 = june_1st.iloc[40:43,:]
product14 = june_1st.iloc[43:46,:]
product15 = june_1st.iloc[46:49,:]
product16 = june_1st.iloc[49:58,:] # 9개
product17 = june_1st.iloc[58:67,:] # 9개
product18 = june_1st.iloc[67:73,:] # 6개
product19 = june_1st.iloc[73:76,:]
product20 = june_1st.iloc[76:79,:]

In [ ]:
# 각 시간대별 예측을 위해 시간 변수를 바꿔서 늘린 테스트 데이터의 마더코드별 데이터셋 구축
product1_total = pd.DataFrame()
product2_total = pd.DataFrame()
product3_total = pd.DataFrame()
product4_total = pd.DataFrame()
product5_total = pd.DataFrame()
product6_total = pd.DataFrame()
product7_total = pd.DataFrame()
product8_total = pd.DataFrame()
product9_total = pd.DataFrame()
product10_total = pd.DataFrame()
product11_total = pd.DataFrame()
product12_total = pd.DataFrame()
product13_total = pd.DataFrame()
product14_total = pd.DataFrame()
product15_total = pd.DataFrame()
product16_total = pd.DataFrame()
product17_total = pd.DataFrame()
product18_total = pd.DataFrame()
product19_total = pd.DataFrame()
product20_total = pd.DataFrame()

In [ ]:
# product1
for i in range(20):
  temp = product1.copy()
  temp['시'] = [(i+6)%24, (i+6)%24, (i+7)%24]
  temp['시간대'] = [str((i + 6)%24) + '시'] * 3
  product1_total = product1_total.append(temp)
  product1_total = product1_total.reset_index(drop=True)

In [ ]:
# product2
for i in range(20):
  temp = product2.copy()
  temp['시'] = [(i+6)%24, (i+6)%24, (i+7)%24]
  temp['시간대'] = [str((i + 6)%24) + '시'] * 3
  product2_total = product2_total.append(temp)
  product2_total = product2_total.reset_index(drop=True)

In [ ]:
# product3
for i in range(20):
  temp = product3.copy()
  temp['시'] = [(i+6)%24, (i+6)%24, (i+7)%24]
  temp['시간대'] = [str((i + 6)%24) + '시'] * 3
  product3_total = product3_total.append(temp)
  product3_total = product3_total.reset_index(drop=True)

In [ ]:
# product4
for i in range(20):
  temp = product4.copy()
  temp['시'] = [(i+6)%24, (i+6)%24, (i+7)%24]
  temp['시간대'] = [str((i + 6)%24) + '시'] * 3
  product4_total = product4_total.append(temp)
  product4_total = product4_total.reset_index(drop=True)

In [ ]:
# product5
for i in range(20):
  temp = product5.copy()
  temp['시'] = [(i+6)%24, (i+6)%24, (i+7)%24]
  temp['시간대'] = [str((i + 6)%24) + '시'] * 3
  product5_total = product5_total.append(temp)
  product5_total = product5_total.reset_index(drop=True)

In [ ]:
# product6
for i in range(20):
  temp = product6.copy()
  temp['시'] = [(i+6)%24, (i+6)%24, (i+7)%24]
  temp['시간대'] = [str((i + 6)%24) + '시'] * 3
  product6_total = product6_total.append(temp)
  product6_total = product6_total.reset_index(drop=True)

In [ ]:
# product7
for i in range(20):
  temp = product7.copy()
  temp['시'] = [(i+6)%24, (i+6)%24, (i+7)%24]
  temp['시간대'] = [str((i + 6)%24) + '시'] * 3
  product7_total = product7_total.append(temp)
  product7_total = product7_total.reset_index(drop=True)

In [ ]:
# product8
for i in range(20):
  temp = product8.copy()
  temp['시'] = [(i+6)%24, (i+6)%24, (i+6)%24, (i+6)%24, (i+7)%24, (i+7)%24]
  temp['시간대'] = [str((i + 6)%24) + '시'] * 6
  product8_total = product8_total.append(temp)
  product8_total = product8_total.reset_index(drop=True)

In [ ]:
# product9
for i in range(20):
  temp = product9.copy()
  temp['시'] = [(i+6)%24]
  temp['시간대'] = [str((i + 6)%24) + '시']
  product9_total = product9_total.append(temp)
  product9_total = product9_total.reset_index(drop=True)

In [ ]:
# product10
for i in range(20):
  temp = product10.copy()
  temp['시'] = [(i+6)%24, (i+6)%24, (i+7)%24]
  temp['시간대'] = [str((i + 6)%24) + '시'] * 3
  product10_total = product10_total.append(temp)
  product10_total = product10_total.reset_index(drop=True)

In [ ]:
# product11
for i in range(20):
  temp = product11.copy()
  temp['시'] = [(i+6)%24, (i+6)%24, (i+7)%24]
  temp['시간대'] = [str((i + 6)%24) + '시'] * 3
  product11_total = product11_total.append(temp)
  product11_total = product11_total.reset_index(drop=True)

In [ ]:
# product12
for i in range(20):
  temp = product12.copy()
  temp['시'] = [(i+6)%24, (i+6)%24, (i+6)%24, (i+6)%24, (i+7)%24, (i+7)%24]
  temp['시간대'] = [str((i + 6)%24) + '시'] * 6
  product12_total = product12_total.append(temp)
  product12_total = product12_total.reset_index(drop=True)

In [ ]:
# product13
for i in range(20):
  temp = product13.copy()
  temp['시'] = [(i+6)%24, (i+6)%24, (i+7)%24]
  temp['시간대'] = [str((i + 6)%24) + '시'] * 3
  product13_total = product13_total.append(temp)
  product13_total = product13_total.reset_index(drop=True)

In [ ]:
# product14
for i in range(20):
  temp = product14.copy()
  temp['시'] = [(i+6)%24, (i+6)%24, (i+7)%24]
  temp['시간대'] = [str((i + 6)%24) + '시'] * 3
  product14_total = product14_total.append(temp)
  product14_total = product14_total.reset_index(drop=True)

In [ ]:
# product15
for i in range(20):
  temp = product15.copy()
  temp['시'] = [(i+6)%24, (i+6)%24, (i+7)%24]
  temp['시간대'] = [str((i + 6)%24) + '시'] * 3
  product15_total = product15_total.append(temp)
  product15_total = product15_total.reset_index(drop=True)

In [ ]:
# product16
for i in range(20):
  temp = product16.copy()
  temp['시'] = [(i+6)%24, (i+6)%24, (i+6)%24, (i+6)%24, (i+6)%24, (i+6)%24, (i+7)%24, (i+7)%24, (i+7)%24]
  temp['시간대'] = [str((i + 6)%24) + '시'] * 9
  product16_total = product16_total.append(temp)
  product16_total = product16_total.reset_index(drop=True)

In [ ]:
# product17
for i in range(20):
  temp = product17.copy()
  temp['시'] = [(i+6)%24, (i+6)%24, (i+6)%24, (i+6)%24, (i+6)%24, (i+6)%24, (i+7)%24, (i+7)%24, (i+7)%24]
  temp['시간대'] = [str((i + 6)%24) + '시'] * 9
  product17_total = product17_total.append(temp)
  product17_total = product17_total.reset_index(drop=True)

In [ ]:
# product18
for i in range(20):
  temp = product18.copy()
  temp['시'] = [(i+6)%24, (i+6)%24, (i+6)%24, (i+6)%24, (i+7)%24, (i+7)%24]
  temp['시간대'] = [str((i + 6)%24) + '시'] * 6
  product18_total = product18_total.append(temp)
  product18_total = product18_total.reset_index(drop=True)

In [ ]:
# product19
for i in range(20):
  temp = product19.copy()
  temp['시'] = [(i+6)%24, (i+6)%24, (i+7)%24]
  temp['시간대'] = [str((i + 6)%24) + '시'] * 3
  product19_total = product19_total.append(temp)
  product19_total = product19_total.reset_index(drop=True)

In [ ]:
# product20
for i in range(20):
  temp = product20.copy()
  temp['시'] = [(i+6)%24, (i+6)%24, (i+7)%24]
  temp['시간대'] = [str((i + 6)%24) + '시'] * 3
  product20_total = product20_total.append(temp)
  product20_total = product20_total.reset_index(drop=True)

In [ ]:
augmented_test = pd.concat([product1_total, product2_total, product3_total, product4_total, product5_total,
                            product6_total, product7_total, product8_total, product9_total, product10_total,
                            product11_total, product12_total, product13_total, product14_total, product15_total,
                            product16_total, product17_total, product18_total, product19_total, product20_total], axis=0)

In [ ]:
augmented_test_fit = augmented_test.iloc[:, 8:-1]

In [ ]:
scaler = RobustScaler()
scaler.fit(augmented_test_fit)
augmented_test_final = pd.DataFrame(scaler.transform(augmented_test_fit), columns=augmented_test_fit.columns)

## 2. 모델에 train으로 학습된 weight를 넣고 합쳐진 test 데이터 prediction

In [ ]:
def build_model(input_layer, start_neurons):
    
    conv1 = Conv1D(start_neurons * 1, 3, activation="relu", padding="same", input_shape=(augmented_test_final.shape[1], 1))(input_layer)
    conv1 = Conv1D(start_neurons * 1, 3, activation="relu", padding="same")(conv1)
    pool1 = BatchNormalization()(conv1)
    pool1 = AveragePooling1D(2)(pool1)
    pool1 = Dropout(0.1)(pool1)

    conv2 = Conv1D(start_neurons * 2, 3, activation="relu", padding="same")(pool1)
    conv2 = Conv1D(start_neurons * 2, 3, activation="relu", padding="same")(conv2)
    pool2 = BatchNormalization()(conv2)
    pool2 = AveragePooling1D(2)(pool2)
    pool2 = Dropout(0.1)(pool2)

    convm = Conv1D(start_neurons * 4, 3, activation="relu", padding="same")(pool2)

    deconv2 = Conv1DTranspose(start_neurons * 1, 3, strides=2, padding="same")(convm)
    uconv2 = concatenate([deconv2, conv2])
    uconv2 = Dropout(0.1)(uconv2)
    uconv2 = Conv1D(start_neurons * 1, 3, activation="relu", padding="same")(uconv2)
    uconv2 = Conv1D(start_neurons * 1, 3, activation="relu", padding="same")(uconv2)
    uconv2 = BatchNormalization()(uconv2)
    uconv2 = Dropout(0.1)(uconv2)

    deconv1 = Conv1DTranspose(start_neurons * 1, 3, strides=2, padding="same")(uconv2)
    uconv1 = concatenate([deconv1, conv1])
    uconv1 = Dropout(0.1)(uconv1)
    uconv1 = Conv1D(start_neurons * 1, 3, activation="relu", padding="same")(uconv1)
    uconv1 = Conv1D(start_neurons * 1, 3, activation="relu", padding="same")(uconv1)
    uconv1 = BatchNormalization()(uconv1)
    uconv1 = Dropout(0.1)(uconv1)

    flat = Flatten()(uconv1)
    dense1 = Dense(256, activation='relu')(flat)
    dense2 = Dense(128, activation='relu')(dense1)
    output_layer = Dense(1)(dense2)
    
    return output_layer

In [ ]:
# model 생성
input_layer = Input((augmented_test_final.shape[1], 1))
output_layer = build_model(input_layer, 512)
model = Model(input_layer, output_layer)

# model 가중치 불러오기
model.load_weights('/content/drive/Shared drives/빅콘테스트2020/weights/mspma/functional_1.50-34.5923.hdf5')

In [ ]:
# 수량 예측
y_pred = model.predict(augmented_test_final.values)

In [ ]:
y_pred = y_pred.reshape(-1)

## 3. 테스트 데이터의 판매단가와 예측된 수량을 곱하여 시간대별 취급액 계산

In [ ]:
augmented_test['취급액'] = y_pred * augmented_test['판매단가']

In [ ]:
# 2020년 6월 1일 각 상품의 시간대별 예상 취급액
mat = pd.pivot_table(augmented_test, index='마더코드', values='취급액', aggfunc='sum', columns='시간대')

In [ ]:
sorted_cols = ['6시', '7시', '8시', '9시', '10시', '11시', '12시', '13시', '14시', '15시',
               '16시', '17시', '18시', '19시', '20시', '21시', '22시', '23시', '0시', '1시']

In [ ]:
mat = mat[sorted_cols]

In [ ]:
mat

시간대,6시,7시,8시,9시,10시,11시,12시,13시,14시,15시,16시,17시,18시,19시,20시,21시,22시,23시,0시,1시
마더코드,,,,,,,,,,,,,,,,,,,,
100012,27321334.80,27282182.24,27282718.33,27318313.61,27394865.68,27534267.43,27666796.14,27659632.90,27660504.74,27612228.06,27214877.85,26453562.82,25700941.91,25080015.93,24713626.54,24517982.92,24401760.30,26192617.50,28714187.75,28444106.45
100080,34999958.75,35173545.21,35571510.34,36018415.59,36374028.86,37047750.22,38086474.10,39056661.54,39321007.22,39266002.46,39498524.99,39645849.90,39828364.49,39702504.76,39763688.48,39842632.44,39848795.40,38947202.32,35995858.22,35757280.20
100148,182913738.55,199100363.82,215778587.64,231154806.11,242904540.12,258973571.75,277227728.38,295778276.77,310039455.16,307498123.01,285319254.87,256124939.71,222862102.19,197398330.25,179574902.35,166315494.34,158619217.96,140001462.57,131123953.70,137015197.50
100150,130787357.73,130873112.13,131208496.17,131970457.32,133070167.14,133812846.95,133745569.14,133078878.17,131968664.77,130271027.30,129123359.16,128665822.43,128777620.29,129095921.71,129262844.27,129568185.95,129966989.99,131273754.49,135833076.52,134824557.86
100205,34584429.15,34468000.64,34398470.36,34227221.60,34053240.23,34019685.69,34160298.38,34840792.02,36401991.81,37815185.25,37936293.12,36885777.35,35100637.07,33589226.79,32303763.62,31462260.08,31105901.23,32856624.93,35770045.49,35610876.65
100320,14866603.02,15383079.77,15921453.61,16501398.79,17089393.71,17603744.47,18152769.55,18758710.28,19263781.94,19401893.28,19092012.50,18657038.39,18246224.26,17906579.95,17661598.24,17538186.05,17484783.72,16538977.07,13717086.79,13769879.43
100348,19761797.96,19895811.14,20202963.16,20397812.28,20501616.80,20594797.42,20710030.15,20780811.80,20553365.27,20203496.94,19859824.39,19437730.74,19014342.87,18685098.07,18523894.26,18507943.10,18512252.39,20278245.12,20438540.64,20189520.03
100362,14891682.31,14906170.23,15039680.22,15270231.25,15526147.77,15711432.61,15793174.15,15685471.06,15372760.54,14849059.34,14392707.96,13963497.72,13694459.18,13403731.21,13091146.10,12778736.03,12509047.87,13137739.53,14471015.73,14579231.14
100381,22638963.32,22906934.56,23166403.66,23366965.03,23541991.65,23644174.83,23621117.60,23103466.78,22417818.20,21826152.70,21366513.39,20723542.49,20272315.28,19892538.80,19720599.21,19603925.30,19546826.67,20846984.61,22693838.29,22570095.47


## 4. MSPMA(Maximum Sales Probability Matching Algorithm) 적용하여 각 상품별 최적 시간대 찾기

In [ ]:
# maxtrix 내 선택을 위한 빈 slot 데이터프레임 만들기
slot = mat.copy()
slot.iloc[:, :] = 0

In [ ]:
# 두 개의 취급액의 한 시간대를 놓고 경합할 경우의 확률적 선택 계산
def probability_matching(column, prod1, prod2):
    sale1 = mat.iloc[prod1, column]
    sale2 = mat.iloc[prod2, column]
    prob_sale1 = round((sale1 + 1e-15) / (sale1 + sale2 + 1e-15), 2)
    prob_sale2 = round((sale2 + 1e-15) / (sale1 + sale2 + 1e-15), 2)
    package = [prod1, prod2]
    return random.choices(package, weights=[prob_sale1, prob_sale2])[0]

In [ ]:
# MSPMA 알고리즘
def mspma(now_row, max_col):
    # 현재 상태의 값을 slot에 표시
    slot.iloc[now_row, max_col] = 1
    
    # slot 테이블에서 max_col에 로우 두 개가 1로 된 경우
    temp = np.array(slot.iloc[:, max_col])
    if len(temp[temp == 1]) == 2:
        
        # now_row외에 다른 1의 row위치를 찾기
        temp_col = list(slot.iloc[:, max_col])
        temp_col[now_row] = -1
        compare_row = np.where(np.array(temp_col) == 1)[0][0]
        
        # 확률 매칭 함수 적용
        result = probability_matching(max_col, now_row, compare_row)
        
        # 비교 대상(compare_row)이 선택되는 경우 (now_row의 max_col 위치가 바뀌어야 하는 상황)
        if result == compare_row:
            # now_row기존의 값을 -1로 대체
            slot.iloc[now_row, max_col] = -1
            mat.iloc[now_row, max_col] = -1
            
            # now_row에 0만 있는 것들 중 가장 큰 것의 인덱스 찾기
            max_col = np.argmax(mat.iloc[now_row, :])

            # slot에 현재 now_row에서 가장 큰 값이라고 표시
            slot.iloc[now_row, max_col] = 1
            
            # 재귀 함수
            mspma(now_row, max_col)

        # 현재 상품(now_row)이 선택되는 경우 (compare_row의 max_col 위치가 바뀌어야 하는 상황)
        else:
            # compare_row기존의 값을 -1로 대체
            slot.iloc[compare_row, max_col] = -1
            mat.iloc[compare_row, max_col] = -1
            
            # compare_row에 0만 있는 것들 중 가장 큰 것의 인덱스 찾기
            max_col = np.argmax(mat.iloc[now_row, :])
            
            # slot에 현재 compare_row에서 가장 큰 값이라고 표시
            slot.iloc[compare_row, max_col] = 1
            
            # 재귀함수
            mspma(compare_row, max_col)

In [ ]:
# 최종 구현
start_row = 0

for i in range(20):
    new_col = np.argmax(mat.iloc[start_row, :])   
    mspma(start_row, new_col)
    start_row += 1

In [ ]:
def isHangul(text):
    pyVer3 =  sys.version_info >= (3, 0)

    if pyVer3 : # for Ver 3 or later
        encText = text
    else: # for Ver 2.x
        if type(text) is not unicode:
            encText = text.decode('utf-8')
        else:
            encText = text

    hanCount = len(re.findall(u'[\u3130-\u318F\uAC00-\uD7A3]+', encText))
    return hanCount > 0

In [ ]:
for i in slot.columns:
    mother_code = slot[i][slot[i] == 1].index[0]
    name = june_1st[june_1st['마더코드'] == mother_code]['상품명']
    
    cnt = 0
    temp = set(name.values)
    for j in temp:
      for k in range(len(j)):
            temp2 = j[k]
            if isHangul(temp2):
                cnt += 2
            else:
                cnt += 1
            
    print('*')
    if len(i) > 2:
        print('--------  ', '------------------', ' '+'-'*cnt+'--'+'-'*(4*len(temp)-1)+'-------')
        print('|', i+' |','-> 마더코드:', mother_code,'| 상품:', set(name.values),'|')
        print('--------  ', '------------------', ' '+'-'*cnt+'--'+'-'*(4*len(temp)-1)+'-------')
    else:
        print('-------  ', '------------------', ' '+'-'*cnt+'--'+'-'*(4*len(temp)-1)+'------')
        print('|', i+' |','-> 마더코드:', mother_code,'| 상품:', str(temp)+'|')
        print('-------  ', '------------------', ' '+'-'*cnt+'--'+'-'*(4*len(temp)-1)+'------')

*
-------   ------------------  --------------------------------------------
| 6시 | -> 마더코드: 100554 | 상품: {'DB손해보험 참좋은운전자보험(1912)'}|
-------   ------------------  --------------------------------------------
*
-------   ------------------  ------------------------------------------------
| 7시 | -> 마더코드: 100555 | 상품: {'디비손보 아임오케이 암보험(20.05런칭)'}|
-------   ------------------  ------------------------------------------------
*
-------   ------------------  -----------------------------------------------
| 8시 | -> 마더코드: 100362 | 상품: {'에이유플러스 슈퍼선스틱 1004(최저가)'}|
-------   ------------------  -----------------------------------------------
*
-------   ------------------  ---------------------------------------
| 9시 | -> 마더코드: 100537 | 상품: {'[기간]제주바다자연산돔39마리'}|
-------   ------------------  ---------------------------------------
*
--------   ------------------  ----------------------------------------
| 10시 | -> 마더코드: 100381 | 상품: {'바비리스 퍼펙트 볼륨스타일러'} |
--------   ------------------  ---------

## * 참고: 기존의 편성표 취급액 합과 MSPMA 적용 후 편성표 취급액 합 비교

기존의 취급액 합

In [ ]:
sclr = RobustScaler()
sclr.fit(june_1st.iloc[:, 8:])
test_scaled = pd.DataFrame(scaler.transform(june_1st.iloc[:, 8:]), columns=june_1st.iloc[:, 8:].columns)

In [ ]:
# model 생성
input_layer = Input((test_scaled.shape[1], 1))
output_layer = build_model(input_layer, 512)
model = Model(input_layer, output_layer)

# model 가중치 불러오기
model.load_weights('/content/drive/Shared drives/빅콘테스트2020/weights/mspma/functional_1.50-34.5923.hdf5')

In [ ]:
# 수량 예측
y_pred = model.predict(test_scaled.values)

In [ ]:
y_pred = y_pred.reshape(-1)

In [ ]:
sum(june_1st['판매단가'] * y_pred)

642569547.9828835

MSPMA 이후 취급액 합

In [ ]:
new_sum = 0

In [ ]:
for i in slot.columns:
    mother_code = slot[i][slot[i] == 1].index[0]
    new_sum += mat[i][mother_code]

In [ ]:
new_sum

811232518.2867279